In [2]:
# Pull yfinance price data for listings in the tsx constituent db table
# Import packages
import yfinance as yf
import os
import pandas as pd
from dotenv import load_dotenv

# Import utilities
from etl.utils.utils import extract_query, load_query

In [5]:
load_dotenv()

# Create db connection string
db_username = os.getenv("DB_USERNAME")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")
db_conn_str = f"mysql+pymysql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}"

In [6]:
# Import current constituent list - get both ticker and id
import_query = """
SELECT ticker
FROM tickers
WHERE `active` = 'Y'
"""

ticker_list = extract_query(sql_query=import_query, db_conn_str=db_conn_str)

In [7]:
# Set start date and end date - These would be set by airflow
start_date = input("Enter start date (yyyy-mm-dd): ").strip()
end_date = input("Enter end date (yyyy-mm-dd): ").strip()

In [8]:
# Create list for batch exports
dfs = []

In [9]:
i = 0
size = 50
while i < len(ticker_list):
    _tickers = yf.Tickers(ticker_list['ticker'].iloc[i:i+size].to_list())
    df = _tickers.history(start = start_date, end = end_date)
    df = df.stack(level=1, future_stack=True).reset_index()
    dfs.append(df)
    i += size

[*********************100%***********************]  50 of 50 completed
[*********************100%***********************]  50 of 50 completed
[*********************100%***********************]  50 of 50 completed
[*********************100%***********************]  50 of 50 completed
[*********************100%***********************]  18 of 18 completed


In [10]:
# Combine dfs
combined_df = pd.concat(dfs, ignore_index=True)

In [11]:
combined_df

Price,Date,Ticker,Close,Dividends,High,Low,Open,Stock Splits,Volume
0,2025-11-03,AAUC.TO,21.799999,0.0,22.139999,21.219999,21.500000,0.0,576400
1,2025-11-03,AAV.TO,11.520000,0.0,11.640000,11.050000,11.100000,0.0,612700
2,2025-11-03,ABX.TO,46.251278,0.0,46.559949,45.454701,45.803202,0.0,7547400
3,2025-11-03,AC.TO,18.850000,0.0,19.080000,18.410000,18.410000,0.0,3780200
4,2025-11-03,ACO-X.TO,52.222553,0.0,52.519778,51.687546,52.182922,0.0,97500
...,...,...,...,...,...,...,...,...,...
4355,2025-11-28,WN.TO,95.528381,0.0,96.156396,95.279168,95.937087,0.0,192100
4356,2025-11-28,WPK.TO,44.080002,0.0,44.259998,43.889999,44.240002,0.0,17100
4357,2025-11-28,WPM.TO,153.470001,0.0,153.960007,150.699997,151.520004,0.0,784800
4358,2025-11-28,WSP.TO,244.449997,0.0,246.100006,244.100006,246.100006,0.0,247700


In [18]:
# Rename columns
combined_df.columns = combined_df.columns.str.lower().str.replace(" ","_")

In [19]:
# Reorder columns
combined_df = combined_df[["date", "ticker", "open", "high", "low", "close", "dividends", "stock_splits", "volume"]]

In [20]:
combined_df

Price,date,ticker,open,high,low,close,dividends,stock_splits,volume
0,2025-11-03,AAUC.TO,21.500000,22.139999,21.219999,21.799999,0.0,0.0,576400
1,2025-11-03,AAV.TO,11.100000,11.640000,11.050000,11.520000,0.0,0.0,612700
2,2025-11-03,ABX.TO,45.803202,46.559949,45.454701,46.251278,0.0,0.0,7547400
3,2025-11-03,AC.TO,18.410000,19.080000,18.410000,18.850000,0.0,0.0,3780200
4,2025-11-03,ACO-X.TO,52.182922,52.519778,51.687546,52.222553,0.0,0.0,97500
...,...,...,...,...,...,...,...,...,...
4355,2025-11-28,WN.TO,95.937087,96.156396,95.279168,95.528381,0.0,0.0,192100
4356,2025-11-28,WPK.TO,44.240002,44.259998,43.889999,44.080002,0.0,0.0,17100
4357,2025-11-28,WPM.TO,151.520004,153.960007,150.699997,153.470001,0.0,0.0,784800
4358,2025-11-28,WSP.TO,246.100006,246.100006,244.100006,244.449997,0.0,0.0,247700


In [ ]:
# Load data to database table
load_query(table_name="price_history", df=combined_df, append=True, db_conn_str=db_conn_str)

In [12]:
# Try to pull history with one ticker that is wrong and another that is right
tickers = yf.Tickers(["TD.TO","TD.TOZO"])

In [13]:
tickers.history()

HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: TD.TOZO"}}}
$TD.TOZO: possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")
[*********************100%***********************]  2 of 2 completed

1 Failed download:
['TD.TOZO']: possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")


Price      Adj Close       Close         Dividends        High          \
Ticker       TD.TOZO       TD.TO TD.TOZO     TD.TO       TD.TO TD.TOZO   
Date                                                                     
2025-12-01       NaN  116.959999     NaN       0.0  117.709999     NaN   
2025-12-02       NaN  118.199997     NaN       0.0  118.220001     NaN   
2025-12-03       NaN  117.699997     NaN       0.0  118.820000     NaN   
2025-12-04       NaN  120.089996     NaN       0.0  120.410004     NaN   
2025-12-05       NaN  122.199997     NaN       0.0  122.239998     NaN   
2025-12-08       NaN  122.150002     NaN       0.0  122.760002     NaN   
2025-12-09       NaN  123.470001     NaN       0.0  123.570000     NaN   
2025-12-10       NaN  125.879997     NaN       0.0  126.190002     NaN   
2025-12-11       NaN  126.430000     NaN       0.0  126.449997     NaN   
2025-12-12       NaN  125.800003     NaN       0.0  126.720001     NaN   
2025-12-15       NaN  126.750000     NaN       0.0  126.919998     NaN   
2025-12-16       NaN  126.110001     NaN       0.0  126.669998     NaN   
2025-12-17       NaN  125.779999     NaN       0.0  126.570000     NaN   
2025-12-18       NaN  127.489998     NaN       0.0  128.240005     NaN   
2025-12-19       NaN  127.989998     NaN       0.0  128.679993     NaN   
2025-12-22       NaN  127.980003     NaN       0.0  128.899994     NaN   
2025-12-23       NaN  128.919998     NaN       0.0  129.169998     NaN   
2025-12-24       NaN  129.160004     NaN       0.0  129.710007     NaN   
2025-12-29       NaN  129.589996     NaN       0.0  129.949997     NaN   
2025-12-30       NaN  129.809998     NaN       0.0  130.169998     NaN   
2025-12-31       NaN  129.360001     NaN       0.0  130.139999     NaN   

Price              Low                Open         Stock Splits    Volume  \
Ticker           TD.TO TD.TOZO       TD.TO TD.TOZO        TD.TO     TD.TO   
Date                                                                        
2025-12-01  116.190002     NaN  117.559998     NaN          0.0   4762200   
2025-12-02  116.150002     NaN  117.459999     NaN          0.0   5311400   
2025-12-03  117.419998     NaN  118.199997     NaN          0.0   3334900   
2025-12-04  116.599998     NaN  117.680000     NaN          0.0   6961000   
2025-12-05  120.510002     NaN  120.790001     NaN          0.0   3842900   
2025-12-08  121.709999     NaN  122.000000     NaN          0.0   4066900   
2025-12-09  122.150002     NaN  122.309998     NaN          0.0   3491500   
2025-12-10  123.730003     NaN  123.779999     NaN          0.0   7435400   
2025-12-11  125.059998     NaN  125.910004     NaN          0.0   7986100   
2025-12-12  125.370003     NaN  126.550003     NaN          0.0   3041100   
2025-12-15  126.000000     NaN  126.000000     NaN          0.0   3083300   
2025-12-16  126.029999     NaN  126.389999     NaN          0.0   5675800   
2025-12-17  124.559998     NaN  126.559998     NaN          0.0   7862200   
2025-12-18  126.000000     NaN  126.519997     NaN          0.0   3541400   
2025-12-19  127.699997     NaN  127.949997     NaN          0.0  11977400   
2025-12-22  127.720001     NaN  128.250000     NaN          0.0   5514200   
2025-12-23  127.970001     NaN  127.970001     NaN          0.0   4915900   
2025-12-24  128.800003     NaN  129.000000     NaN          0.0   1117200   
2025-12-29  129.160004     NaN  129.160004     NaN          0.0   7057400   
2025-12-30  129.589996     NaN  129.770004     NaN          0.0   5139300   
2025-12-31  129.160004     NaN  129.809998     NaN          0.0   2305203   

Price               
Ticker     TD.TOZO  
Date                
2025-12-01     NaN  
2025-12-02     NaN  
2025-12-03     NaN  
2025-12-04     NaN  
2025-12-05     NaN  
2025-12-08     NaN  
2025-12-09     NaN  
2025-12-10     NaN  
2025-12-11     NaN  
2025-12-12     NaN  
2025-12-15     NaN  
2025-12-16     NaN  
2025-12-17     NaN  
2025-12-18     NaN  
2025-12-19     Na